In [ ]:
# -------------------------------------------------------------------------------------
# Third party imports
# -------------------------------------------------------------------------------------
import numpy as np 
import matplotlib.pyplot as plt
import os, sys
import glob 

import astropy
from astropy.io import fits
from astropy.time import Time
from astropy.units import allclose as quantity_allclose
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates.builtin_frames import FK5, ICRS, GCRS, GeocentricMeanEcliptic, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, GeocentricTrueEcliptic, BarycentricTrueEcliptic, HeliocentricTrueEcliptic, HeliocentricEclipticIAU76
from astropy.constants import R_sun, R_earth
from astropy.nddata import CCDData

from matplotlib.colors import LogNorm

from ccdproc import Combiner
from ccdproc import wcs_project

import inspect
import importlib

# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------
sys.path.append(os.path.join(os.path.split(os.getcwd())[0], 'shifty'))
import downloader 
import refcat
import imagehandler
import known

importlib.reload(downloader)
importlib.reload(refcat)
importlib.reload(imagehandler)
importlib.reload(known)

# OneImage
This class exists for dealing with individual images. 
Currently its functionality is pretty limited, so I'm not convinced it needs to be a seperate class, as the functionality could just get rolled in to ImageEnsemble.
But maybe we'll add more to it that will be valuable to have on a per-image basis.

Start by initializing the class with a filename. 

In [ ]:
filename = '../dev_data/2015RS281_HSC20160826_123.fits'
one=imagehandler.OneImage(filename)

The initialization can be given additional keywords:

        extno          - int          - Extension number of image data
                                      - (0 if single-extension)
                                      - default is 0
        verbose        - bool         - Print extra stuff if True
                                      - default is False
        EXPTIME        - str OR float - Exposure time in seconds
                                      - (keyword or value)
                                      - default is 'EXPTIME'
        MAGZERO        - str OR float - Zeropoint magnitude
                                      - (keyword or value)
                                      - default is 'MAGZERO'
        MJD_START      - str OR float - MJD at start of exposure
                                      - (keyword or value)
                                      - default is 'MJD-STR'
        GAIN           - str OR float - Gain value (keyword or value)
                                      - default is 'GAINEFF'
        FILTER         - str          - Filter name (keyword)
                                      - default is 'FILTER'
        NAXIS1         - str OR int   - Number of pixels along axis 1
                                      - default is 'NAXIS1'
        NAXIS2         - str OR int   - Number of pixels along axis 2
                                      - default is 'NAXIS2'
        INSTRUMENT     - str          - Instrument name (keyword)
                                      - default is 'INSTRUME'
Most of this is probably not actually needed, but I figured that for now it would be good to have handy any header values that we _might_ need. 

The _one_ object should now have a few attributes:

In [ ]:
[key for key in one.__dict__.keys()]

- _readOneImageAndHeader_ is the function used for reading a file. It's available here as a pseudo-method, in case a new file should get read in or the object was initialized without a filename.
- _header_keywords_ is a dictionary connecting the specified names of keywords to their default names.
- _key_values_ is a dictionary with the values of the header keywords
- _WCS_ is the WCS
- _header_ is the entire header of the image extension
- _header0_ is the entire main header (multi-extension fits often have some info, like WCS, in individual extension headers and other info, like exposure time, in the main header in extension 0).
- _data_ is a 2D numpy array with the pixel data
- _filename_ remembers the input filename
- _extno_ remembers the input extension number

In [ ]:
print(f"one.extno: {one.extno}")
print(f"one.filename: {one.filename}")
print(f"Data shape: {one.data.shape}, 3x3 cutout:")
print(one.data[81:84,81:84])
print("")
print(one.WCS)
print(f"\none.header_keywords content:")
_ = [print(o) for o in one.header_keywords.items()]
print(f"\none.key_values content:")
_ = [print(o) for o in one.key_values.items()]
plt.imshow((one.data-one.data.min())[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()

# DataEnsemble
This class is used for accumulating information about a whole set of associated images. 
It contains these methods:
- reproject_data - for aligning (by reprojection) images to a shared WCS. Optional step.
It is essentially a container for: a 3D data array, an array of WCS objects, and an array of header objects. This can be used for loading 

In [ ]:
# I'm gonna define this flag, so that it's easy to switch between
# using both nights (24 images) and just the second night (12 images).
# Only the last 12 images are in GitHub repo.
both_nights=False
#both_nights=True

First make a list of filenames:

In [ ]:
importlib.reload(imagehandler)
filename = '../dev_data/2015RS281_HSC20160826_112.fits'
if both_nights:
    filenames=[filename.replace('112', str(i)) for i in np.arange(100, 124)]
    filenames[:12]=[filenamei.replace('0826', '0825') for filenamei in filenames[:12]]
else:
    filenames=[filename.replace('112', str(i)) for i in np.arange(112, 124)]

filenames

Initialize with the list of filenames:

In [ ]:
importlib.reload(imagehandler)
E=imagehandler.DataEnsemble(filenames)

The initialization can be given the same additional keywords as OneImage. Additionally, _extno_ is allowed to be a list/array, in case the desired image isn't in the same extension for all images.

The content of _E_ looks much the same as for _one_, but most of the attributes are now arrays (if they weren't already) with the additional dimension of having an entry per image.

In [ ]:
[key for key in E.__dict__.keys()]

_E_ also have _reprojected_ attribute starting as False.

If neccessary, the data arrays can be reprojected (using ccdproc.wcs_project) so that the arrays have the same shared projection.
This is slow (several minutes), and isn't neccessary if you're only interested in a small part of the field and the fact that the images are offset/warped relative to each other gets built into the shifts.
The HSC images used here were reprojected previously, so we'll skip this here. If this step is run, the _.reprojected_ attribute gets set to _True_ and _.data_ is overwritten with the reprojected data. (Note to self: prevent _.reproject_data_ from running if _.reprojected==True_)
It is probably unneccessary to reproject in most cases, as the offset between images can be accounted for using the calculated shifts later, so reprojection only makes sense if the curvature is large, in which case getting all the images on the same projection is useful.

In [ ]:
%%time
#E.reproject_data() # Slow!
print(E.reprojected)

# DataHandler
This class is used for handling the data. It contains several DataEnsemble objects within it, and methods for manipulating them.  
These methods exist:
- integer_shift - shift image data by an integer number of pixels. 
- stack - stacks all the data arrays, either shifted or non-shifted (keyword controlled).
- save_stack - saves a stacked array to a fits file.
- save_shift - saves the shifted arrays to fits files.

In [ ]:
# I'm gonna define this flag, so that it's easy to switch between
# using both nights (24 images) and just the second night (12 images).
# Only the last 12 images are in GitHub repo.
both_nights=False
#both_nights=True
#Then define a bunch of filenames:
importlib.reload(imagehandler)
filename = '../dev_data/2015RS281_HSC20160826_112.fits'
if both_nights:
    filenames=[filename.replace('100', str(i)) for i in np.arange(100, 124)]
    filenames[-12:]=[filenamei.replace('0825', '0826') for filenamei in filenames[-12:]]
else:
    filenames=[filename.replace('112', str(i)) for i in np.arange(112, 124)]

filenames

In [ ]:
importlib.reload(imagehandler)
D=imagehandler.DataHandler(filenames)

The initialization can be given the same additional keywords as DataEnsemble. 

The content of _D_ is the list of filenames, the list of extension numbers, and then three DataEnsemble objects for storing the _image_data_ , _shifted_data_ and _stacked_data_, which all look like the DataEnsemble object _E_ above, although the latter two start out empty. 

In [ ]:
print('D:',[key for key in D.__dict__.keys()])
print('D.image_data:', [key for key in D.image_data.__dict__.keys()])
print('D.shifted_data:', [key for key in D.stacked_data.__dict__.keys()])
print('D.stacked_data:', [key for key in D.stacked_data.__dict__.keys()])

Let's stack the non-shifted images, both mean and median stacking. Mean is better as it preserves photometric accuracy, but sometimes median is nice to look at. 

In [ ]:
%%time
#D.stack(shifted=False, save_to_filename='mean_stack.fits')
D.stack(shifted=False)
print(D.stacked_data.data.min())
plt.imshow((D.stacked_data.data-D.stacked_data.data.min())[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()

In [ ]:
%%time
#D.stack(shifted=False, median_combine=True, save_to_filename='median_stack.fits')
D.stack(shifted=False, median_combine=True)
plt.imshow((D.stacked_data.data-D.stacked_data.data.min())[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()

Now define some an array of shifts and shift the data. _.integer_shift_ can take a _padmean=True_ keyword to pad the shifted data with the mean value instead of NaN.

In [ ]:
#Different shifts whether we're using 12 or 24 images:
if both_nights:
    shifts=np.array([620, 505]) - np.array([np.concatenate([np.linspace(253, 329, 12),
                                                            np.linspace(542, 620, 12)]),
                                            np.concatenate([np.linspace(288, 332, 12),
                                                            np.linspace(459, 505, 12)])]
                                           ).round(0).astype(int).T
else:
    shifts=np.array([620, 505]) - np.array([np.linspace(542, 620, 12),
                                            np.linspace(459, 505, 12)]
                                           ).round(0).astype(int).T
print(shifts)
D.integer_shift(shifts, padmean=True)
#D.integer_shift(shifts, padmean=False)

_D.shifted_data_ now contains the shifted image array, which is slightly larger than the original so as to not cut out anything:

In [ ]:
print(D.image_data.data.shape, D.image_data.WCS.shape, np.shape(D.image_data.header))
print(D.shifted_data.data.shape, D.shifted_data.WCS.shape, np.shape(D.shifted_data.header))

Now stack and save, both mean and median stacking. Again, mean is better for photometric properties, but median removes more of the star signal.

In [ ]:
%%time
#D.stack(shifted=True, save_to_filename='shifted_mean_stack.fits')
D.stack(shifted=True)
plt.imshow((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()
print(np.max((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605]))

In [ ]:
%%time
#D.stack(shifted=True, median_combine=True, save_to_filename='shifted_median_stack.fits')
D.stack(shifted=True, median_combine=True)
plt.imshow((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()
print(np.max((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605]))



This is just an experiment. One can subtrack the mean non-shifted image from each image before shifting and median-combining for even better cleaning. 

In [ ]:
%%time
D=imagehandler.DataHandler(filenames)
D.stack(shifted=False, median_combine=False)
D.image_data.data=np.array([D.image_data.data[i]-D.stacked_data.data for i in np.arange(len(D.image_data.data))])
D.integer_shift(shifts, padmean=True)
D.stack(shifted=True, median_combine=True, save_to_filename='shifted_median_stack_clean.fits')
plt.imshow((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()
print(np.max((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605]))

#### Shift & Stacking a known object!
Here is a demonstration of the D._calculate_shifts_from_known method and the D.shift_stack_by_name method.

First, a demo of how D.calculate_shifts_from_known can be used to calculate the shifts for a known object (2015 RS281, a TNO), which can then be used to shift and stack the images:

In [ ]:
%%time
importlib.reload(imagehandler)
D=imagehandler.DataHandler(filenames)
shifts = D._calculate_shifts_from_known(object_name='2015 RS281', obs_code='568')
print(shifts)
D.integer_shift(shifts, padmean=True)
D.stack(shifted=True, median_combine=True)
plt.imshow((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()

Or, even simpler, with the D.shift_stack_by_name method:

In [ ]:
%%time
importlib.reload(imagehandler)
D=imagehandler.DataHandler(filenames)
D.stack(shifted=False, median_combine=False)
D.image_data.data=np.array([D.image_data.data[i]-D.stacked_data.data for i in np.arange(len(D.image_data.data))])
D.shift_stack_by_name('2015 RS281', obs_code='568', padmean=True, median_combine=True)
D.save_stack('shift+stack_along_2015_RS281.fits')
plt.imshow((D.stacked_data.data-np.nanmin(D.stacked_data.data))[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()

Here's a different object (2008 SV52, an asteroid):

In [ ]:
%%time
importlib.reload(imagehandler)
D=imagehandler.DataHandler(filenames)
D.stack(shifted=False, median_combine=False)
D.image_data.data=np.array([D.image_data.data[i]-D.stacked_data.data for i in np.arange(len(D.image_data.data))])
D.shift_stack_by_name('2008 SV52', obs_code='568', padmean=True, median_combine=True)
D.save_stack('shift+stack_along_2008_SV52.fits')
plt.imshow((D.stacked_data.data-np.nanmin(D.stacked_data.data))[3122:3322, 2088:2288], origin='lower', cmap='gray',norm=LogNorm())
cbar = plt.colorbar()

# Source Extractor Wrapper in Python (SEWPy)
Just a little playing around with _sewpy_. This should not go here in the end.

In [ ]:
import sewpy
import importlib
importlib.reload(sewpy)
sew = sewpy.SEW(sexpath='sex',
                params=["X_IMAGE", "Y_IMAGE", "FLUX_AUTO", "FLUXERR_AUTO", "SNR_WIN",
                        "FWHM_IMAGE", "ELONGATION", "ISOAREAF_IMAGE", "FLAGS", "NUMBER"],
                config={"DETECT_MINAREA":10, "DETECT_THRESH":2.5, "ANALYSIS_THRESH":1.5})

In [ ]:
out = sew("shifted_median_stack_clean.fits")
out["table"] # this is an astropy table.

So, it's clearly pretty easy to get a SourceExtractor catalogue that has a very limited number of detections, of which only the known object has FLAGS==0. These are the flag bits:

    1	aperture photometry is likely to be biased by neighboring sources or
        by more than 10% of bad pixels in any aperture
    2	the object has been deblended
    4	at least one object pixel is saturated
    8	the isophotal footprint of the detected object is truncated (too close to an image boundary)
    16	at least one photometric aperture is incomplete or corrupted (hitting buffer or memory limits)
    32	the isophotal footprint is incomplete or corrupted (hitting buffer or memory limits)
    64	a memory overflow occurred during deblending
    128	a memory overflow occurred during extraction

So FLAGS==3 means both flags 1+2 apply. Anything with 2 or 3 is certainly bad, as it indicates it has found multiple sources right up against each other, so probably star/galaxy noise. I think anything with a 2 (or sum of 2 + other flag) can safely be ignored. Also, the detections can be filtered by ELONGATION and FWHM

In [ ]:
idx=[(bin(o['FLAGS'])[-2]!='1') & (o['ELONGATION']<2) for o in out['table']]
good_sources=out['table'][idx]
good_sources

There, easy peasy, get the source and nothing else :-)

















# Junk & testing
These things are here mostly as a reminder to myself about some of the functionality of various packages/classes that might be helpful to take advantage of. 

In [ ]:
# A CCDData object can easily be constructed, as so:
dat = CCDData(one.data, wcs=one.WCS, header=one.header, unit='adu')
dat.header

In [ ]:
# wcs_reproject can take larger orders and a target_shape. 
# target_shape might be useful for preserving all the data, rather than trimming it to fit
# inside the shape of the target WCS. Need to investigate
reprojected_image=wcs_project(dat, E.WCS[0], order='bilinear', target_shape=(5000,5000))

In [ ]:
E.WCS[0].all_pix2world([1,2,3], [1,1,1], 1)

In [ ]:
# The centre pixels of a WCS can easily be changed, like so:
# This might be useful for propagating the WCS to the shift+stacked images,
# as those have a margin added to them, changing the center coordinate.
print(E.WCS[2])
E.WCS[1].wcs.crpix=(100,100)
print(E.WCS[2])
# NAXIS might also need to be changed.

In [ ]:
E.stack(shifted=True, median_combine=True, save_to_filename='tst.fits')

In [ ]:
%%time
E.save_stack()

ie. we are not limited by writing files to disk. The shifting and especially the stacking is the time consuming part.

In [ ]:
%%time
#E.stack(shifted=False, save_to_filename='mean_stack.fits')
E.stack(shifted=False)

In [ ]:
%%time
stack = E.data.mean(0)
print(np.shape(stack))
#plt.imshow((E.stacked_data-E.stacked_data.min())[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
#cbar = plt.colorbar()

In [ ]:
%%time
stack = np.median(E.data, 0)
print(np.shape(stack))
#plt.imshow((E.stacked_data-E.stacked_data.min())[520:720,405:605], origin='lower', cmap='gray',norm=LogNorm())
#cbar = plt.colorbar()

In [ ]:
rep=[CCDData(E.data[0], unit='adu'), CCDData(E.data[1], unit='adu'), CCDData(E.data[2], unit='adu')]

In [ ]:
np.array(CCDData(E.data[0], unit='adu'))

In [ ]:
CCDData(E.data[0], unit='adu').data

In [ ]:
CCDData(E.data, wcs1=E.WCS, unit='adu')

In [ ]:
help(CCDData)

In [ ]:
E.MJD.shape

In [ ]:
E.extno

In [ ]:
one.header['SIMPLE']

In [ ]:
one.header['ABCD']=True

In [ ]:
#one.header.append(('ABCDEf', False, 'test'),end=False)
one.header['COMMENT']=('S_MJD was set from MJD_STR')
one.header['S_MJD']=('asdfg','hjkl')
one.header

In [ ]:
one.header['asdfghjkl']

In [ ]:
E.WCS.shape

In [ ]:
one.header_keywords.items()

In [ ]:
one.key_values

In [ ]:
for key, use in one.header_keywords.items():
    print()
    print(f'SHIFTY_{key} derived from {use}')

In [ ]:
for i in one.header_keywords.items():
    print(i)

In [ ]:
one.header['S_MJD']=one.header['CUNIT2A']

In [ ]:
one.header.comments['NAXIS1']

In [ ]:
from astropy import wcs
wcs.WCS(one.header)

In [ ]:
one.header['COMMENTS']='asda'

In [ ]:
one.header

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.today())

In [ ]:
    (f'SHIFTY_MJD_MID derived from '
     f'{one.header_keywords["MJD_START"]} and {one.header_keywords["EXPTIME"]}.')

In [ ]:
one.header

In [ ]:
one.header['COMMENT']

In [ ]:
imagehandler.ImageEnsemble()

In [ ]:
if None:
    print(1)

In [ ]:
target=10
len(f'Data reprojected to WCS of file {target} at {str(datetime.datetime.today())[:19]}')

In [ ]:
shifts.shape

In [ ]:
def save_stack(DataEnsembleObject, filename='stack.fits'):
        '''
        Save a stack to a fits file.
        '''
        print(f'Saving stack to file {filename}')
        hdu = fits.PrimaryHDU(DataEnsembleObject.data)
        hdu.writeto(filename, overwrite=True)
        print('Done!')

In [ ]:
hdu=fits.PrimaryHDU(data=E.shifted_data[0], header=one.header)
hdu.writeto('test.fits', overwrite=True)

In [ ]:
help(fits.PrimaryHDU)

In [ ]:
E.WCS[0].to_header()

In [ ]:
one.header['BP_7_1']

In [ ]:
help(one.WCS)

In [ ]:
one.WCS.to_header(relax=True)['CD*']

In [ ]:
help(hdu)

In [ ]:
hdu[0].header['CD1_1']

In [ ]:
one.header.update(E.WCS[5].to_header(relax=True))

In [ ]:
wcs.WCS(one.header)

In [ ]:
E.WCS[5].to_header(relax=True)['CD*']

In [ ]:
one.header['CD*']

In [ ]:
hdu[0].header['CD*']

In [ ]:
one.header0.update(hdu[0].header)

In [ ]:
for i in one.WCS.items():
 print(i)

In [ ]:
one.header0['CD*']

In [ ]:
E.WCS[0].to_header(relax=True)

In [ ]:
one.header['CD*_*']

In [ ]:
del one.header['CD2_1']

In [ ]:
one.header['CD*']

In [ ]:
hdu=E.WCS[5].to_fits(relax=True)

In [ ]:
hdu[0].header['CD*']

In [ ]:
import copy
two=copy.deepcopy(one)

In [ ]:
two.header['CD*']

In [ ]:
two.header.update(E.WCS[5].to_header(relax=True))

In [ ]:
two.header['CD*']

In [ ]:
one.header['CD*']

In [ ]:
del one.header['CD*']

In [ ]:
one.header['CD?_?']

In [ ]:
h=E.WCS[5]

In [ ]:
h=E.WCS[4]

In [ ]:
E.WCS[4]

In [ ]:
E.WCS[4].wcs.crpix

In [ ]:
E.WCS[4].wcs.crpix += (1,2)

In [ ]:
E.WCS[4].wcs.crpix

In [ ]:
now = str(datetime.today())[:19]

In [ ]:
i=5
ymax = shifts[:, 0].max()
xmax = shifts[:, 1].max()
pad_size = ((shifts[i, 0], ymax - shifts[i, 0]),  # Size of pad
                        (shifts[i, 1], xmax - shifts[i, 1]))  # on 4 sides

com_str = (f'At {now} data was padded by {pad_size}')

In [ ]:
len(com_str)

In [ ]:
int(len(E.WCS)/2)

In [ ]:
isinstance('123',str)

In [ ]:
which_WCS='drgd'
wcsidx=0 if which_WCS.lower()=='first' else -1 if which_WCS.lower()=='last' else 5
print(wcsidx)

In [ ]:
len(E.data)

In [ ]:
np.shape(E.WCS) == ()

In [ ]:
filename='data.fits'

In [ ]:
'fits' in filename

In [ ]:
len(one.data.shape)==2
x=5

In [ ]:
filename.replace('.fits',f'_{1234:05.0f}.fits')

In [ ]:
help(fits.PrimaryHDU)

In [ ]:
one.header[:]=one.header['SIMPLE']

In [ ]:
one.header[:5]